<a href="https://colab.research.google.com/github/a-ayushigit/Recommendation_movies/blob/main/MatrixFactorizationAnd_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

In [2]:
df = pd.read_csv('/content/movie_new_df.csv')

In [3]:
pt = df.pivot_table(index='userId',columns='title',values='rating')

In [4]:
pt.fillna(0 , inplace=True)

In [5]:
pt.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),...,Young Guns (1988),Zack and Miri Make a Porno (2008),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#create a transpose
movie_user_pt = pt.T

In [7]:
movie_user_pt.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
"10,000 BC (2008)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101 Dalmatians (1996),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


In [38]:
movie_user_sp_matrix = csr_matrix(movie_user_pt.values)

In [39]:
movie_user_sp_matrix

<1635x538 sparse matrix of type '<class 'numpy.float64'>'
	with 72210 stored elements in Compressed Sparse Row format>

In [40]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(movie_user_sp_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [45]:
np.random.choice(movie_user_pt.shape[0])

921

In [46]:
query_index = np.random.choice(movie_user_pt.shape[0])

In [49]:
type(movie_user_pt.iloc[query_index,:])

pandas.core.series.Series

In [50]:
distances , indices = model.kneighbors(movie_user_pt.iloc[query_index,:].values.reshape(1,-1) ,n_neighbors=6 )

In [51]:
distances

array([[1.11022302e-16, 3.13600048e-01, 4.57262706e-01, 4.63995095e-01,
        4.86075532e-01, 5.25985016e-01]])

In [52]:
indices

array([[1322,  351,  106,  170,  917, 1488]])

In [73]:
def recommend(movie):
  movie_index = np.where(movie_user_pt.index == movie)[0][0]
  distances , indices = model.kneighbors(movie_user_pt.iloc[movie_index,:].values.reshape(1,-1) ,n_neighbors=6 )
  data = []
  for i in indices.flatten()[1:]:  # Skip the first movie (itself)
        data.append(movie_user_pt.index[i])
  return data


In [105]:
recommend('Kung Fu Panda (2008)')

['Sherlock Holmes (2009)',
 'Despicable Me (2010)',
 'Avatar (2009)',
 'Iron Man (2008)',
 'Zombieland (2009)']

In [78]:
movie_user_pt

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
"10,000 BC (2008)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101 Dalmatians (1996),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zoolander (2001),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0
Zootopia (2016),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
eXistenZ (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0


In [80]:
movie_user_pt.shape

(1635, 538)

In [81]:
SVD = TruncatedSVD(n_components=12 , random_state=17)

In [82]:
matrix = SVD.fit_transform(movie_user_pt)

In [83]:
matrix.shape

(1635, 12)

In [84]:
corr = np.corrcoef(matrix)

In [85]:
corr.shape

(1635, 1635)

In [89]:
movies_list = list(pt.columns)

In [100]:
def recommend_SVD(movie):
  movie_index = movies_list.index(movie)
  corr_movie = corr[movie_index]
  p = list(sorted(enumerate(corr_movie) , key=lambda x:x[1] , reverse=True))
  data = []
  for i in p[1:10] :
    data.append(movies_list[i[0]])
  return data


In [104]:
recommend_SVD('Kung Fu Panda (2008)')

['Sherlock Holmes (2009)',
 'Despicable Me (2010)',
 'Toy Story 3 (2010)',
 'Thor (2011)',
 'Avatar (2009)',
 'X-Men: First Class (2011)',
 'Megamind (2010)',
 'Harry Potter and the Deathly Hallows: Part 1 (2010)',
 'Captain America: The First Avenger (2011)']